In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
import numpy as np
import seaborn as sns

### 서울의 주택 공급률을 연도별로 정리해서 가져와보자 
### https://stat.molit.go.kr/portal/cate/viewChk.do?hRsId=29&hFormId=2100 주택보급률 신버젼

In [2]:
options = Options()
prefs = {'download.default_directory': '/Users/dw/ws/eda/data', 'download.prompt_for_download': False}
options.add_experimental_option('prefs', prefs)
service = Service("../driver/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# 주택보급률 신버젼 URL
url = "https://stat.molit.go.kr/portal/cate/viewChk.do?hRsId=29&hFormId=2100"
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

In [3]:
years = [str(year) for year in range(2005, 2023)]
data = []
# 연도 순차적으로 선택
for year in years:
    # 시작 드롭다운에서 연도 선택
    dropdown_element = driver.find_element(By.XPATH, "//select[@id='sStart']")
    dropdown = Select(dropdown_element)
    dropdown.select_by_value(year)
    time.sleep(1)
    # 종료 드롭다운에서 연도 선택
    dropdown_element2 = driver.find_element(By.XPATH, "//select[@id='sEnd']")
    dropdown2 = Select(dropdown_element2)
    dropdown2.select_by_value(year)
    time.sleep(1)
    # 버튼 클릭
    button = driver.find_element(By.XPATH, "//button[@onclick='javascript:goformView();']")
    button.click()
    time.sleep(1)
    # HTML 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 연도에 따라 target_rows 설정
    if 2005 <= int(year) <= 2009:
        target_rows = ['3', '2', '5']  # 3은 서울, 2는 수도권, 5는 대구
    else:
        target_rows = ['2', '4', '6']  # 2는 수도권, 4는 서울, 6은 대구
    
    rows = soup.find_all('tr', class_='GMDataRow')
    for row in rows:
        # onmousemove 속성에서 target_rows에 있는 번호 확인
        if any(f'Grids[0].Rows["{row_num}"]' in row.get('onmousemove', '') for row_num in target_rows):
            cells = row.find_all('td', class_='GMClassReadOnly')  # 'GMClassReadOnly' 클래스를 가진 셀 찾기
            if cells:  # 셀이 존재할 경우
                values = [cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)]  # 텍스트 추출
                if len(values) == 6:
                    # 지역 이름을 연도에 따라 설정
                    if 2005 <= int(year) <= 2009:
                        if 'Grids[0].Rows["3"]' in row.get('onmousemove', ''):
                            region_name = "서울"
                        elif 'Grids[0].Rows["2"]' in row.get('onmousemove', ''):
                            region_name = "수도권"
                        elif 'Grids[0].Rows["5"]' in row.get('onmousemove', ''):
                            region_name = "대구"
                    else:
                        if 'Grids[0].Rows["2"]' in row.get('onmousemove', ''):
                            region_name = "수도권"
                        elif 'Grids[0].Rows["4"]' in row.get('onmousemove', ''):
                            region_name = "서울"
                        elif 'Grids[0].Rows["6"]' in row.get('onmousemove', ''):
                            region_name = "대구"
                    
                    data.append((year, region_name, values))  # 연도, 지역 이름과 데이터를 튜플로 추가

# DataFrame으로 변환
df = pd.DataFrame(data, columns=['Year', 'Region', 'Values'])


In [35]:
print(df)

    Year Region                                            Values      가구수  \
0   2005    수도권           [7,462.1, 7,165.0, 96.0, 0.0, 0.0, 0.0]  7,462.1   
1   2005     서울           [3,309.9, 3,102.2, 93.7, 0.0, 0.0, 0.0]  3,309.9   
2   2005     대구               [814.6, 779.7, 95.7, 0.0, 0.0, 0.0]    814.6   
3   2006    수도권           [7,628.2, 7,302.0, 95.7, 0.0, 0.0, 0.0]  7,628.2   
4   2006     서울           [3,349.7, 3,151.1, 94.1, 0.0, 0.0, 0.0]  3,349.7   
5   2006     대구               [817.4, 793.7, 97.1, 0.0, 0.0, 0.0]    817.4   
6   2007    수도권           [7,813.0, 7,419.1, 95.0, 0.0, 0.0, 0.0]  7,813.0   
7   2007     서울           [3,403.2, 3,172.3, 93.2, 0.0, 0.0, 0.0]  3,403.2   
8   2007     대구               [823.4, 816.9, 99.2, 0.0, 0.0, 0.0]    823.4   
9   2008    수도권           [7,994.4, 7,624.7, 95.4, 0.0, 0.0, 0.0]  7,994.4   
10  2008     서울           [3,453.6, 3,231.7, 93.6, 0.0, 0.0, 0.0]  3,453.6   
11  2008     대구              [829.3, 853.9, 103.0, 0.0, 0.0, 0.0

### values를 분리

In [36]:
df2 = df[['가구수', '주택수', '보급률', '가구수new', '주택수new', '보급률new']] = pd.DataFrame(df['Values'].tolist(), index=df.index)
df2 = df.drop(columns=['Values'])
print(df2)

    Year Region      가구수      주택수    보급률    가구수new    주택수new 보급률new
0   2005    수도권  7,462.1  7,165.0   96.0       0.0       0.0    0.0
1   2005     서울  3,309.9  3,102.2   93.7       0.0       0.0    0.0
2   2005     대구    814.6    779.7   95.7       0.0       0.0    0.0
3   2006    수도권  7,628.2  7,302.0   95.7       0.0       0.0    0.0
4   2006     서울  3,349.7  3,151.1   94.1       0.0       0.0    0.0
5   2006     대구    817.4    793.7   97.1       0.0       0.0    0.0
6   2007    수도권  7,813.0  7,419.1   95.0       0.0       0.0    0.0
7   2007     서울  3,403.2  3,172.3   93.2       0.0       0.0    0.0
8   2007     대구    823.4    816.9   99.2       0.0       0.0    0.0
9   2008    수도권  7,994.4  7,624.7   95.4       0.0       0.0    0.0
10  2008     서울  3,453.6  3,231.7   93.6       0.0       0.0    0.0
11  2008     대구    829.3    853.9  103.0       0.0       0.0    0.0
12  2009    수도권  8,168.9  7,795.5   95.4       0.0       0.0    0.0
13  2009     서울  3,500.9  3,258.4   93.1       0

In [37]:
print(df2.dtypes)

Year      object
Region    object
가구수       object
주택수       object
보급률       object
가구수new    object
주택수new    object
보급률new    object
dtype: object


In [38]:
# 쉼표 제거 후 데이터 타입 변환
df2['Year'] = df2['Year'].astype(int)
df2['가구수'] = df2['가구수'].str.replace(',', '').astype(float)
df2['주택수'] = df2['주택수'].str.replace(',', '').astype(float)
df2['보급률'] = df2['보급률'].str.replace(',', '').astype(float)
df2['가구수new'] = df2['가구수new'].str.replace(',', '').astype(float)
df2['주택수new'] = df2['주택수new'].str.replace(',', '').astype(float)
df2['보급률new'] = df2['보급률new'].str.replace(',', '').astype(float)
# 변환 후 데이터 타입 확인
print(df2.dtypes)


Year        int64
Region     object
가구수       float64
주택수       float64
보급률       float64
가구수new    float64
주택수new    float64
보급률new    float64
dtype: object


In [39]:
condition = (df2['가구수new'] == 0.0) & (df2['주택수new'] == 0.0) & (df2['보급률new'] == 0.0)

df2.loc[condition, '가구수new'] = df2['가구수']
df2.loc[condition, '주택수new'] = df2['주택수']
df2.loc[condition, '보급률new'] = df2['보급률']

In [40]:
print(df2)

    Year Region     가구수     주택수    보급률   가구수new   주택수new  보급률new
0   2005    수도권  7462.1  7165.0   96.0   7462.1   7165.0    96.0
1   2005     서울  3309.9  3102.2   93.7   3309.9   3102.2    93.7
2   2005     대구   814.6   779.7   95.7    814.6    779.7    95.7
3   2006    수도권  7628.2  7302.0   95.7   7628.2   7302.0    95.7
4   2006     서울  3349.7  3151.1   94.1   3349.7   3151.1    94.1
5   2006     대구   817.4   793.7   97.1    817.4    793.7    97.1
6   2007    수도권  7813.0  7419.1   95.0   7813.0   7419.1    95.0
7   2007     서울  3403.2  3172.3   93.2   3403.2   3172.3    93.2
8   2007     대구   823.4   816.9   99.2    823.4    816.9    99.2
9   2008    수도권  7994.4  7624.7   95.4   7994.4   7624.7    95.4
10  2008     서울  3453.6  3231.7   93.6   3453.6   3231.7    93.6
11  2008     대구   829.3   853.9  103.0    829.3    853.9   103.0
12  2009    수도권  8168.9  7795.5   95.4   8168.9   7795.5    95.4
13  2009     서울  3500.9  3258.4   93.1   3500.9   3258.4    93.1
14  2009     대구   834.6  

In [41]:
df2 = df2.drop(columns=['가구수', '주택수', '보급률'])

In [42]:
print(df2)

    Year Region   가구수new   주택수new  보급률new
0   2005    수도권   7462.1   7165.0    96.0
1   2005     서울   3309.9   3102.2    93.7
2   2005     대구    814.6    779.7    95.7
3   2006    수도권   7628.2   7302.0    95.7
4   2006     서울   3349.7   3151.1    94.1
5   2006     대구    817.4    793.7    97.1
6   2007    수도권   7813.0   7419.1    95.0
7   2007     서울   3403.2   3172.3    93.2
8   2007     대구    823.4    816.9    99.2
9   2008    수도권   7994.4   7624.7    95.4
10  2008     서울   3453.6   3231.7    93.6
11  2008     대구    829.3    853.9   103.0
12  2009    수도권   8168.9   7795.5    95.4
13  2009     서울   3500.9   3258.4    93.1
14  2009     대구    834.6    871.1   104.4
15  2010    수도권   8527.1   8216.1    96.4
16  2010     서울   3646.5   3442.1    94.4
17  2010     대구    873.0    885.9   101.5
18  2011    수도권   8655.9   8382.7    96.8
19  2011     서울   3673.4   3477.8    94.7
20  2011     대구    883.7    894.9   101.3
21  2012    수도권   8788.8   8555.3    97.3
22  2012     서울   3700.6   3509.8 

In [43]:
# 열 이름에서 'new' 제거
df2 = df2.rename(columns={
    '가구수new': '가구수',
    '주택수new': '주택수',
    '보급률new': '보급률'
})

In [44]:
print(df2)

    Year Region      가구수      주택수    보급률
0   2005    수도권   7462.1   7165.0   96.0
1   2005     서울   3309.9   3102.2   93.7
2   2005     대구    814.6    779.7   95.7
3   2006    수도권   7628.2   7302.0   95.7
4   2006     서울   3349.7   3151.1   94.1
5   2006     대구    817.4    793.7   97.1
6   2007    수도권   7813.0   7419.1   95.0
7   2007     서울   3403.2   3172.3   93.2
8   2007     대구    823.4    816.9   99.2
9   2008    수도권   7994.4   7624.7   95.4
10  2008     서울   3453.6   3231.7   93.6
11  2008     대구    829.3    853.9  103.0
12  2009    수도권   8168.9   7795.5   95.4
13  2009     서울   3500.9   3258.4   93.1
14  2009     대구    834.6    871.1  104.4
15  2010    수도권   8527.1   8216.1   96.4
16  2010     서울   3646.5   3442.1   94.4
17  2010     대구    873.0    885.9  101.5
18  2011    수도권   8655.9   8382.7   96.8
19  2011     서울   3673.4   3477.8   94.7
20  2011     대구    883.7    894.9  101.3
21  2012    수도권   8788.8   8555.3   97.3
22  2012     서울   3700.6   3509.8   94.8
23  2012     대구 

In [1]:
# 2015년부터 2020년까지 데이터 필터링
filtered_df = df2[(df2['Year'] >= 2005) & (df2['Year'] <= 2022)]
# 피벗 테이블로 데이터 변환
pivot_df = filtered_df.pivot(index='Year', columns='Region', values='보급률')
# 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(pivot_df.index, pivot_df['서울'], marker='o', label='수도권')

plt.title('2005-2022년 서울 주택수 공급량')
plt.xlabel('년도')
plt.ylabel('주택수')
plt.xticks(pivot_df.index)  # x축 눈금 설정
plt.legend()
plt.grid()
plt.show()

NameError: name 'df2' is not defined

In [ ]:
import mysql.connector
import pandas as pd

# connect_database 함수 정의
def connect_database(host_name="host_name", user_name="user_name", user_password="user_password"):
    mydb = mysql.connector.connect(
        host=host_name,
        port=3306,
        user=user_name,
        password=user_password,
        database="WAY_BACK_HOME"
    )
    cursor = mydb.cursor()
    return mydb, cursor

# 데이터베이스 연결
mydb, cursor = connect_database()

# 테이블 생성 (없을 경우)
create_table_query = """
CREATE TABLE IF NOT EXISTS HOUSE_SUPPLY (
    Year INT,
    Region VARCHAR(10),
    가구수 FLOAT,
    주택수 FLOAT,
    보급률 FLOAT
);
"""
cursor.execute(create_table_query)
# DataFrame df2를 SQL INSERT 문으로 변환하여 저장
for index, row in df2.iterrows():
    sql = "INSERT INTO HOUSE_SUPPLY (Year, Region, 가구수, 주택수, 보급률) VALUES (%s, %s, %s, %s, %s)"
    cursor.execute(sql, (row['Year'], row['Region'], row['가구수'], row['주택수'], row['보급률']))

# 변경 사항을 커밋하고 연결 종료
mydb.commit()
cursor.close()
mydb.close()